## Request and Inspect Weekly SPX-Options

First, load ib-ruby environment

In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Check the Presence of predefined indices

In [2]:
Symbols::Index.all

[:a_d, :asx, :dax, :hsi, :minihsi, :spx, :stoxx, :tick, :trin, :vasx, :vdax, :vhsi, :vix, :volume, :vstoxx]

> Symbols::Index holds a selection of predefined indices

In [3]:
G.logger.level = Logger::FATAL
spx_market_price =  Symbols::Index.spx.market_price


4231.78

_Note:_ If the request is fired before the market opening, the returned market price is the closing price of the previous day.

 * Get the most recent expiry 
 * Define a range of strikes to consider as  market-price +/- 15 points.
 


In [4]:
# declare variables to be modified in the loop
spxw_options=  nil
date =  DateTime.now

loop do  
 spxw_options =  Option.new( trading_class: 'SPXW', 
                             symbol: 'SPX', 
                             currency: 'USD', 
                             expiry: date.strftime("%Y%m%d").to_i,
                             exchange: 'SMART', 
                             right: :put ).verify do |c|
      c if c.strike > spx_market_price - 15 && c.strike < spx_market_price + 15 
   end
 if spxw_options.empty?
   date = date.tomorrow
 else
   break
 end
end
  
puts spxw_options.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 491690717 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4220.0 │   USD    │
│ Option │ SPX    │ 491690721 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4225.0 │   USD    │
│ Option │ SPX    │ 491690724 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 491690731 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4240.0 │   USD    │
│ Option │ SPX    │ 495223431 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4235.0 │   USD    │
│ Option │ SPX    │ 495223432 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4245.0 │   USD    │
└

Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks 


In [5]:
spxw_options.map{|c| c.request_greeks thread: true }.each &:join
puts spxw_options.greek.as_table

┌────────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                         │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ put  SPX/SPXW 20210616  4220.0 │    0.00 │       0.05 │     0.00 │   -0.00 │    0.00 │    0.00 │   -0.00 │
│ put  SPX/SPXW 20210616  4225.0 │    0.61 │       0.04 │     0.00 │   -0.76 │    0.43 │    0.06 │   -0.10 │
│ put  SPX/SPXW 20210616  4230.0 │    5.51 │       0.04 │     0.00 │   -1.00 │    0.00 │    0.00 │    0.01 │
│ put  SPX/SPXW 20210616  4240.0 │   15.51 │       0.05 │     0.00 │   -1.00 │    0.00 │    0.00 │    0.01 │
│ put  SPX/SPXW 20210616  4235.0 │   10.51 │       0.04 │     0.00 │   -1.00 │    0.00 │    0.00 │    0.01 │
│ put  SPX/SPXW 20210616  4245.0 │   20.51 │       0.07 │     0.00 │   -1.00 │    0.00 │    0.00 │    0.01 │
└──────────────────

### Get ATM-Weeklies

There is a method `Contract#atm-options`. Its designed for monthly options

In [6]:
atm_options =  Symbols::Index.spx.atm_options

atm_options.keys    # Expiries of the Options

[2106, 2107, 2108, 2109, 2111, 2112, 2201, 2202, 2203, 2205, 2206, 2212, 2301]

In [7]:
puts atm_options[atm_options.keys.first].as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 467200278 │  SMART   │ 20210617 │    100     │      SPX      │  put  │ 4230.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


To get all available ATM-SPXW-Options we define a sample (as above) and specify the strike

In [8]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, strike: 4230, 
                   exchange: :smart, right: :put

spxw_chain =  spxw.verify.sort_by &:last_trading_day
puts spxw_chain.as_table


┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ SPX    │ 491690724 │  SMART   │ 20210616 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 467183869 │  SMART   │ 20210618 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 492917794 │  SMART   │ 20210621 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 493489210 │  SMART   │ 20210623 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 483864365 │  SMART   │ 20210625 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│ Option │ SPX    │ 494243330 │  SMART   │ 20210628 │    100     │     SPXW      │  put  │ 4230.0 │   USD    │
│

In [9]:
spxw_chain.map{|o| o.request_greeks( thread: true)}.each &:join
puts spxw_chain.greek.as_table

┌────────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                         │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ put  SPX/SPXW 20210616  4230.0 │    5.62 │       0.03 │     0.00 │   -1.00 │    0.00 │    0.00 │    0.01 │
│ put  SPX/SPXW 20210618  4230.0 │   22.75 │       0.16 │     0.55 │   -0.55 │    0.01 │    1.24 │      -- │
│ put  SPX/SPXW 20210621  4230.0 │   26.91 │       0.12 │     0.79 │   -0.54 │    0.01 │    1.96 │      -- │
│ put  SPX/SPXW 20210623  4230.0 │   32.23 │       0.12 │     0.85 │   -0.53 │    0.01 │    2.33 │      -- │
│ put  SPX/SPXW 20210625  4230.0 │   36.82 │       0.13 │     0.96 │   -0.53 │    0.00 │    2.64 │   -1.85 │
│ put  SPX/SPXW 20210628  4230.0 │   39.38 │       0.12 │     1.34 │   -0.53 │    0.00 │    3.05 │   -1.48 │
│ put  SPX/SPXW 202